# Display Selection Teaching Notebook (Interactive)

This notebook is a **teaching/interactive** tool that walks through:
1) **Library generation** from one or multiple reference backbones (with degeneracy),
2) **Display selection simulation** (mRNA / yeast / yotta),
3) Optional **NGS simulation** from either the true library or a selected round,
and
4) **Visualizations** (e.g., uniform vs log-normal abundance skew).

👉 **How to use**
- Run each cell in order.
- Use the widgets to set parameters (single vs multi reference, sequences, degeneracy, etc.).
- Click the buttons to generate outputs and plots.

👉 **Requirements**
```bash
pip install numpy pandas matplotlib ipywidgets
jupyter nbextension enable --py widgetsnbextension  # classic notebook
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```
(In JupyterLab, ipywidgets works out-of-the-box in current versions.)

👉 **Notes**
- This notebook is **self-contained** (does not require your package). You can copy/paste it anywhere.
- Degeneracy input uses a simple syntax: `pos:chars` pairs, comma-separated (1-based positions).
  Example: `10:N,25:R,42:AC`.

---

In [13]:
import math, json, random
from dataclasses import dataclass
from itertools import product
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

try:
    from ipywidgets import (
        VBox, HBox, Accordion, Tab, Layout,
        Dropdown, Textarea, Text, IntText, FloatText, BoundedFloatText,
        Checkbox, Button, HTML, Accordion, IntSlider, FloatSlider, Label
    )
    from IPython.display import display, Markdown, clear_output
    _WIDGETS_OK = True
except Exception:
    _WIDGETS_OK = False
    print("ipywidgets not available; you can still run the core functions below.")

## Helpers & Core Logic

In [14]:
IUPAC_DNA = {
    "A":"A","C":"C","G":"G","T":"T",
    "R":"AG","Y":"CT","S":"GC","W":"AT","K":"GT","M":"AC",
    "B":"CGT","D":"AGT","H":"ACT","V":"ACG","N":"ACGT"
}
DNA4 = list("ACGT")
AA20 = list("ARNDCEQGHILKMFPSTWYV")

def _strip_fasta(seq_text: str) -> str:
    """
    Accepts plain sequence or FASTA text; returns uppercase sequence without headers/spaces.
    """
    lines = [ln.strip() for ln in seq_text.strip().splitlines() if ln.strip()]
    if not lines:
        return ""
    if lines[0].startswith(">"):
        # skip headers
        lines = [ln for ln in lines if not ln.startswith(">")]
    return "".join(lines).upper()

def parse_deg_string(deg: str) -> List[Tuple[int, str]]:
    """
    Parse a string like: "10:N,25:R,42:AC" into [(10, "N"), (25, "R"), (42, "AC")]
    """
    if not deg.strip():
        return []
    out = []
    parts = [p.strip() for p in deg.split(",") if p.strip()]
    for p in parts:
        if ":" not in p:
            raise ValueError(f"Degeneracy item '{p}' must be of the form pos:chars")
        pos_s, chars = p.split(":", 1)
        pos = int(pos_s.strip())
        out.append((pos, chars.strip().upper()))
    return out

def normalize_degeneracy(seq_type: str, chars: str) -> str:
    chars = chars.upper().replace(" ", "")
    if seq_type.lower() == "dna":
        expanded = []
        for ch in chars:
            if ch in IUPAC_DNA:
                expanded.extend(IUPAC_DNA[ch])
            elif ch in "ACGT":
                expanded.append(ch)
            else:
                raise ValueError(f"Invalid DNA char: {ch}")
        return "".join(sorted(set(expanded)))
    # amino acids
    bad = set(chars) - set(AA20)
    if bad:
        raise ValueError(f"Invalid AA chars: {bad}")
    return "".join(sorted(set(chars)))

def theoretical_max(pos_to_choices: Dict[int, str]) -> int:
    m = 1
    for chs in pos_to_choices.values():
        m *= len(chs)
    return m

def generate_library_from_one(
    backbone_seq: str,
    seq_type: str,
    pos_degeneracy: List[Tuple[int, str]],
    n_sequences: Optional[int] = None,
    seed: int = 0
) -> List[str]:
    """
    Generate sequences from a single backbone.
    If n_sequences is None -> enumerate full combinatorial expansion.
    Else: sample unique sequences up to n_sequences (reproducibly by seed).
    """
    L = len(backbone_seq)
    pos_to_choices = {}
    for pos, chars in pos_degeneracy:
        if not (1 <= pos <= L):
            raise ValueError(f"Position {pos} out of 1..{L}")
        pos_to_choices[pos] = normalize_degeneracy(seq_type, chars)

    tmax = theoretical_max(pos_to_choices)
    rng = random.Random(seed)

    if n_sequences is None:
        # full enumeration
        if not pos_to_choices:
            return [backbone_seq]
        positions = sorted(pos_to_choices.keys())
        choice_lists = [list(pos_to_choices[p]) for p in positions]
        seqs = []
        for combo in product(*choice_lists):
            s = list(backbone_seq)
            for p, ch in zip(positions, combo):
                s[p-1] = ch
            seqs.append("".join(s))
        return seqs

    # sample unique
    n = min(int(n_sequences), tmax if tmax > 0 else int(n_sequences))
    seqs_set = set()
    def mutate():
        s = list(backbone_seq)
        for p, choices in pos_to_choices.items():
            s[p-1] = rng.choice(choices)
        return "".join(s)
    while len(seqs_set) < n:
        seqs_set.add(mutate())
    return list(seqs_set)

def selection_simulate_rounds(
    df_library: pd.DataFrame,
    rounds: int = 3,
    display: str = "yeast",
    mode: str = "competitive",
    target_conc_nM: float = 50.0,
    S: float = 0.7,
    pcr_cycles: int = 10,
    bias_sigma: float = 0.3,
    expr_enabled: bool = True,
    expr_sigma: float = 0.5,
    KD_mu_log10: float = -7.0,
    KD_alpha: float = 1.0,
    KD_sigma: float = 0.3,
    input_molecules: int = 200_000,
    seed: int = 4242
):
    """
    Minimal display-selection simulator (single target for clarity).
    Returns list of per-round DataFrames.
    """
    rng = np.random.default_rng(seed)
    N = len(df_library)
    # start frequencies
    if "start_frequency" not in df_library.columns:
        freq = np.ones(N) / N
    else:
        freq = df_library["start_frequency"].to_numpy()
        freq = freq / freq.sum()

    # expression heterogeneity (yeast-like)
    expr = rng.lognormal(mean=0.0, sigma=expr_sigma, size=N) if expr_enabled else np.ones(N)

    # per-sequence baseline "trait"
    a = rng.normal(0, 1.0, size=N)
    # KD on log10 scale -> numeric KD (relative units)
    log10KD = KD_mu_log10 - KD_alpha * a + rng.normal(0, KD_sigma, size=N)
    KD = 10.0 ** log10KD

    def occupancy(C, KD):
        return C / (C + KD)

    rounds_out = []
    for r in range(1, rounds+1):
        # capture probability ~ S * occupancy * expression
        p_bind = S * occupancy(target_conc_nM, KD) * expr
        eff = p_bind * freq
        if eff.sum() <= 0:
            eff = np.ones_like(eff) / len(eff)
        probs = eff / eff.sum()

        # captured counts (multinomial)
        captured = rng.multinomial(input_molecules, probs)

        # PCR amplification with bias
        G = 2 ** pcr_cycles
        bias = rng.lognormal(mean=0.0, sigma=bias_sigma, size=N)
        lam = captured * G * bias
        amplified = rng.poisson(lam)
        total = max(amplified.sum(), 1)
        freq = amplified / total

        out = df_library.copy()
        out["round"] = r
        out["capture_prob"] = p_bind
        out["captured"] = captured
        out["amplified"] = amplified
        out["frequency"] = freq
        out["KD"] = KD
        rounds_out.append(out)

    return rounds_out

def ngs_from_pool(df_pool: pd.DataFrame, reads_total: int, p_error: float, seed: int = 999):
    """
    Simulate NGS reads from a pool with 'sequence' and 'frequency' columns.
    Returns a DataFrame of reads with error annotations.
    """
    if "frequency" not in df_pool.columns:
        raise ValueError("df_pool must have a 'frequency' column")
    seqs = df_pool["sequence"].astype(str).str.upper().to_numpy()
    probs = df_pool["frequency"].to_numpy()
    probs = probs / probs.sum()
    rng = np.random.default_rng(seed)

    # choose DNA/AA alphabet based on first seq
    is_dna = set(seqs[0]).issubset(set("ACGTN"))
    alpha = list("ACGT") if is_dna else AA20

    # CDF sample
    cdf = np.cumsum(probs)
    def pick_idx(u):
        return np.searchsorted(cdf, u, side="right")

    reads = []
    for i in range(reads_total):
        idx = pick_idx(rng.random())
        src = seqs[idx]
        obs_chars, errs = [], 0
        for ch in src:
            if rng.random() < p_error:
                # flip to a different character in alphabet
                choices = [a for a in alpha if a != ch]
                obs_chars.append(choices[int(rng.integers(0, len(choices)))])
                errs += 1
            else:
                obs_chars.append(ch)
        reads.append({"read_id": f"read_{i:07d}", "source_seq": src, "observed_seq": "".join(obs_chars), "num_errors": errs})
    return pd.DataFrame(reads)

## Interactive Controls

In [15]:
LIB_DF = None
ROUNDS = None
READS_DF = None

In [19]:
# Inline backend + crisp figs
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.dpi"] = 120

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, Markdown
import ipywidgets as W

# helper: pretty DataFrame HTML (no truncation, wraps long seqs)
def render_df(df, max_rows=10):
    html = (
        df.head(max_rows)
          .to_html(index=False, escape=False)
          .replace('<table', '<table style="font-size:12px;border-collapse:collapse;table-layout:fixed;width:100%"')
          .replace('<th>', '<th style="text-align:left;border-bottom:1px solid #ddd;padding:4px 8px;white-space:normal">')
          .replace('<td>', '<td style="text-align:left;padding:4px 8px;vertical-align:top;word-break:break-word;white-space:normal">')
    )
    return f'<div style="max-height:340px;overflow:auto">{html}</div>'

# tiny ⓘ icon with hover text
def info_icon(text):
    return W.HTML(value=f"<span title='{text}' style='cursor:help;font-weight:bold'>ⓘ</span>")

# ---- OUTPUT AREAS (plots/text land here) ----
out_gen_txt   = W.HTML()
out_gen_fig   = W.Output()
out_sel_txt   = W.HTML()
out_sel_fig   = W.Output()
out_ngs_txt   = W.HTML()
out_ngs_fig   = W.Output()
out_ab_txt    = W.HTML()
out_ab_fig    = W.Output()

# ---- WIDGETS (wide + non-truncated labels) ----
wide = {'width':'760px'}
label_style = {'description_width':'initial'}

# Library
dd_mode   = W.Dropdown(options=[("Single reference","single"), ("Multiple references","multi")],
                       value="single", description="Mode:", layout=W.Layout(width='260px'),
                       style=label_style, tooltip="Choose 1 or 2 reference backbones")
ta_seqA   = W.Textarea(
    value=">A\nATGCGTACGTGCTAGCTAGCTGATCGATGCTAGCTAGGCTAGCTAGCTAGGATCCGATCGTAGCTAGCTAGCTAGCTA",
    description="Backbone A:", layout=W.Layout(**wide, height='80px'), style=label_style,
    tooltip="FASTA or raw sequence")
txt_degA  = W.Text(value="10:N,25:R,42:AC", description="Deg A (pos:chars):",
                   layout=W.Layout(width='380px'), style=label_style,
                   tooltip="Comma list like 10:N,25:R (1-based positions)")
int_nA    = W.IntText(value=500, description="Sample N(A):", layout=W.Layout(width='180px'), style=label_style)
float_wA  = W.FloatText(value=1.0, description="Weight A:", layout=W.Layout(width='180px'), style=label_style)

ta_seqB   = W.Textarea(
    value=">B\nATGCGTACGATCGTAGCTAGCTAGGCTAGCTAGGATCCGATCGTAGCTAGCTAGCTAGCTAGCGTACGTAGCTAGCTA",
    description="Backbone B:", layout=W.Layout(**wide, height='80px'), style=label_style)
txt_degB  = W.Text(value="8:AC,40:N", description="Deg B (pos:chars):",
                   layout=W.Layout(width='380px'), style=label_style)
int_nB    = W.IntText(value=500, description="Sample N(B):", layout=W.Layout(width='180px'), style=label_style)
float_wB  = W.FloatText(value=0.5, description="Weight B:", layout=W.Layout(width='180px'), style=label_style)

cb_full      = W.Checkbox(value=False, description="Enumerate full combinatorial (ignore N samples)",
                          style=label_style, tooltip="If checked, enumerate all combinations")
int_seed_lib = W.IntText(value=12345, description="Seed (library):", layout=W.Layout(width='220px'), style=label_style)
btn_gen      = W.Button(description="Generate Library", button_style="success", layout=W.Layout(width='200px'))

# Selection
cb_enable_sel = W.Checkbox(value=True, description="Enable selection", style=label_style)

dd_display = W.Dropdown(
    options=[("Yeast", "yeast"), ("mRNA", "mrna"), ("Yotta", "yotta")],  # UPDATED
    value="yeast",
    description="Display",
    style=label_style,
    tooltip="Choose the display technology; presets will adjust relevant knobs."
)
int_rounds    = W.IntSlider(value=3, min=1, max=6, step=1, description="Rounds", style=label_style)
float_S       = W.BoundedFloatText(value=0.7, min=0, max=1, step=0.05, description="Stringency S", style=label_style,
                                   tooltip="Scales capture probability (higher → stricter)")
float_C       = W.BoundedFloatText(value=50.0, min=0.01, max=1e6, step=1.0, description="Target C (nM)", style=label_style)
cb_expr       = W.Checkbox(value=True, description="Expression heterogeneity", style=label_style,
                           tooltip="Cell-to-cell variation; favors high expressers when >0")
float_expr    = W.BoundedFloatText(value=0.5, min=0, max=2.0, step=0.05, description="Expr σ", style=label_style)
int_cycles    = W.IntSlider(value=10, min=0, max=20, step=1, description="PCR cycles", style=label_style)
float_bias    = W.BoundedFloatText(value=0.3, min=0, max=2.0, step=0.05, description="PCR bias σ", style=label_style)
float_muKD    = W.FloatText(value=-7.0, description="μ(log10 KD)", style=label_style)
float_alphaKD = W.FloatText(value=1.0, description="α (affinity spread)", style=label_style)
float_sigmaKD = W.FloatText(value=0.3, description="σ KD", style=label_style)
int_input     = W.IntText(value=200000, description="Molecules/round", style=label_style)
int_seed_sel  = W.IntText(value=4242, description="Seed (select)", style=label_style)
btn_sel       = W.Button(description="Run Selection", button_style="info")

# NGS
cb_enable_ngs = W.Checkbox(value=True, description="Enable NGS", style=label_style)
dd_ngs_source  = W.Dropdown(options=[("From selection round","selection"), ("From true library (uniform)","library")],
                            value="selection", description="NGS source", style=label_style)
int_ngs_round  = W.IntSlider(value=3, min=1, max=6, step=1, description="Use round", style=label_style)
int_reads      = W.IntText(value=10000, description="Reads total", style=label_style)
float_perr     = W.BoundedFloatText(value=0.001, min=0, max=0.2, step=0.0005, description="Per‑base error p", style=label_style)
int_seed_ngs   = W.IntText(value=999, description="Seed (NGS)", style=label_style)
btn_ngs        = W.Button(description="Simulate NGS", button_style="warning")

# Abundance viz
dd_abund  = W.Dropdown(options=[("Uniform","uniform"), ("Log-normal (σ=1.0)","ln1"), ("Log-normal (σ=0.5)","ln0.5")],
                       value="uniform", description="Abundance model", style=label_style)
btn_abund = W.Button(description="Show abundance …")

# ---- Tooltips for labels (fix 'null' hovers) ----
def _set_tip(widget, text):
    # v8+ uses .tooltip; v7 used .description_tooltip
    if hasattr(widget, "tooltip"):
        widget.tooltip = text
    else:  # fallback for very old ipywidgets
        try:
            widget.description_tooltip = text
        except Exception:
            pass             # fallback

def attach_tooltips():
    # Library
    _set_tip(dd_mode, "Use one or two reference backbones.")
    _set_tip(ta_seqA, "Reference sequence A (FASTA or raw; '>' header lines ignored).")
    _set_tip(txt_degA, "Degeneracy for A: comma list of 1‑based pos:chars. Example 10:N,25:R,42:AC")
    _set_tip(int_nA, "If full enumeration is OFF, sample this many unique A variants.")
    _set_tip(float_wA, "Relative abundance weight for backbone A when mixing A and B.")
    _set_tip(ta_seqB, "Reference sequence B (optional if Mode=Single).")
    _set_tip(txt_degB, "Degeneracy for B: comma list of 1‑based pos:chars. Example 8:AC,40:N")
    _set_tip(int_nB, "If full enumeration is OFF, sample this many unique B variants.")
    _set_tip(float_wB, "Relative abundance weight for backbone B when mixing A and B.")
    _set_tip(cb_full, "If checked, enumerate ALL combinations from degeneracy and ignore Sample N(A/B).")
    _set_tip(int_seed_lib, "Random seed for reproducible library sampling.")
    _set_tip(btn_gen, "Generate the ground‑truth library and show preview + length histogram.")

    # Selection
    _set_tip(cb_enable_sel, "Toggle the selection simulator on/off.")
    _set_tip(dd_display, "Preset label for display type; simulator logic is generic.")
    _set_tip(int_rounds, "Number of selection cycles (bind→wash→PCR).")
    _set_tip(float_S, "Stringency S (0–1). Probability cutoff for survival. Higher = harsher filtering (tight binders survive).")
    _set_tip(float_C, "Target (ligand) concentration in nM. Higher = easier for weak binders; lower favors only strong binders.")
    _set_tip(cb_expr, "Include cell‑to‑cell expression heterogeneity.")
    _set_tip(float_expr, "Expression noise σ (0=uniform; higher → some clones over/under-express).")
    _set_tip(int_cycles, "PCR cycles per round (amplification depth).")
    _set_tip(float_bias, "PCR bias σ per round (skew / jackpotting).")
    _set_tip(float_muKD, "Mean log10 KD across variants (e.g., −7 ≈ 100 nM).")
    _set_tip(float_alphaKD, "Affinity spread parameter α (larger → broader KD spectrum).")
    _set_tip(float_sigmaKD, "Per‑variant KD noise σ (measurement/realization variability).")
    _set_tip(int_input, "Total molecules sampled per round (bottleneck).")
    _set_tip(int_seed_sel, "Random seed for the selection simulation.")
    _set_tip(btn_sel, "Run selection rounds and plot diversity/trajectories.")

    # NGS
    _set_tip(cb_enable_ngs, "Toggle NGS read simulation on/off.")
    _set_tip(dd_ngs_source, "Read from a selection round (skewed) or from the true library (uniform).")
    _set_tip(int_ngs_round, "Which selection round to use as the NGS source.")
    _set_tip(int_reads, "Total number of reads to simulate.")
    _set_tip(float_perr, "Per‑base substitution probability (e.g., 0.001 ≈ Q30).")
    _set_tip(int_seed_ngs, "Random seed for the NGS simulation.")
    _set_tip(btn_ngs, "Simulate reads and plot error‑count histogram.")

    # Abundance viz
    _set_tip(dd_abund, "Compare even (uniform) vs skewed (log‑normal) starting abundances.")
    _set_tip(btn_abund, "Draw abundance histogram and top‑K cumulative mass curve.")

# ---- DISPLAY PRESETS: apply when tech changes (PUT HERE after widgets) ----
def _apply_display_presets(kind: str):
    """
    Set sensible defaults when user switches display tech.
    """
    if kind == "mrna":
        cb_expr.value       = False
        float_expr.value    = 0.0
        int_rounds.value    = 4
        float_C.value       = 50.0
        float_S.value       = 0.7
        int_input.value     = 1_000_000_000   # 1e9
        float_bias.value    = 0.2
        float_sigmaKD.value = 0.2
    elif kind == "yeast":
        cb_expr.value       = True
        float_expr.value    = 0.5
        int_rounds.value    = 3
        float_C.value       = 25.0
        float_S.value       = 0.7
        int_input.value     = 10_000_000      # 1e7
        float_bias.value    = 0.3
        float_sigmaKD.value = 0.3
    elif kind == "yotta":  # barcode-on-yeast; dampen some noise
        cb_expr.value       = True
        float_expr.value    = 0.3
        int_rounds.value    = 3
        float_C.value       = 25.0
        float_S.value       = 0.7
        int_input.value     = 100_000_000     # 1e8
        float_bias.value    = 0.2
        float_sigmaKD.value = 0.2

# call once at startup
_apply_display_presets(dd_display.value)

# re-apply when dropdown changes
def _on_display_change(change):
    if change["name"] == "value":
        _apply_display_presets(change["new"])

dd_display.observe(_on_display_change, names="value")

# Help lines (hover ⓘ and short blurbs)
help_lib = W.HTML(
    "Backbone: reference sequence. Deg: <code>pos:chars</code> (1‑based). "
    "Sample N: how many variants to draw (unchecked = enumerate all). Weight: mix ratio when combining A/B."
)
help_sel = W.HTML(
    "S: stricter selection ↑. Target C: higher makes binding easier. "
    "Expr σ: expression variability (off for mRNA; on for Yeast; lower for Yotta). "
    "PCR cycles + bias σ: amplification & skew. KD params (μ, α, σ) shape affinity."
)
help_ngs = W.HTML(
    "NGS source: selected pool (skewed) vs true library (uniform). Per‑base error p: substitution chance per position."
)
help_ab  = W.HTML("Log‑normal starts uneven; uniform is flat. See how top‑K mass behaves.")

# Data holders
LIB_DF, ROUNDS, READS_DF = None, None, None

# ---- LAYOUT / RENDER ----
lib_box = W.VBox([
    W.HBox([dd_mode, info_icon("Use one or two reference backbones.")]),
    ta_seqA, W.HBox([txt_degA, int_nA, float_wA]),
    ta_seqB, W.HBox([txt_degB, int_nB, float_wB]),
    W.HBox([cb_full, int_seed_lib, btn_gen]),
    help_lib, out_gen_txt, out_gen_fig
])
sel_box = W.VBox([
    W.HBox([cb_enable_sel, dd_display, int_rounds, float_S, float_C]),
    W.HBox([cb_expr, float_expr, int_cycles, float_bias]),
    W.HBox([float_muKD, float_alphaKD, float_sigmaKD, int_input, int_seed_sel, btn_sel]),
    help_sel, out_sel_txt, out_sel_fig
])
ngs_box = W.VBox([
    W.HBox([cb_enable_ngs, dd_ngs_source, int_ngs_round]),
    W.HBox([int_reads, float_perr, int_seed_ngs, btn_ngs]),
    help_ngs, out_ngs_txt, out_ngs_fig
])
viz_box = W.VBox([
    W.HBox([dd_abund, btn_abund, info_icon("Compare even vs skewed starting abundance.")]),
    help_ab, out_ab_txt, out_ab_fig
])

# attach tooltip text to labels BEFORE rendering
attach_tooltips()

display(Markdown("### 1) Library generation")); display(lib_box)
display(Markdown("### 2) Selection (display)"));  display(sel_box)
display(Markdown("### 3) NGS simulation"));       display(ngs_box)
display(Markdown("### 4) Visualization: abundance skew (uniform vs. log‑normal)")); display(viz_box)

# ---- CALLBACKS (write to Output panes) ----
def on_generate(_):
    global LIB_DF, ROUNDS
    ROUNDS = None
    with out_gen_fig:
        clear_output(wait=True)
        try:
            mode = dd_mode.value
            seqA = _strip_fasta(ta_seqA.value)
            degA = parse_deg_string(txt_degA.value)
            nA   = None if cb_full.value else int_nA.value
            wA   = float_wA.value

            recs = []
            seqsA = generate_library_from_one(seqA, "dna", degA, n_sequences=nA, seed=int_seed_lib.value)
            for i, s in enumerate(seqsA):
                recs.append({"seq_id": f"A_{i:05d}", "sequence": s, "backbone_id":"A", "start_weight": wA})

            if mode == "multi":
                seqB = _strip_fasta(ta_seqB.value)
                degB = parse_deg_string(txt_degB.value)
                nB   = None if cb_full.value else int_nB.value
                wB   = float_wB.value
                seqsB = generate_library_from_one(seqB, "dna", degB, n_sequences=nB, seed=int_seed_lib.value+101)
                for i, s in enumerate(seqsB):
                    recs.append({"seq_id": f"B_{i:05d}", "sequence": s, "backbone_id":"B", "start_weight": wB})

            df = pd.DataFrame(recs)
            df["start_frequency"] = df["start_weight"] / df["start_weight"].sum()
            LIB_DF = df

            out_gen_txt.value = f"<b>✅ Generated library with {len(df)} sequences.</b>" + render_df(
                df[["seq_id","sequence","backbone_id","start_weight","start_frequency"]]
            )

            fig, ax = plt.subplots(figsize=(6,3))
            ax.hist(df["sequence"].str.len(), bins=10)
            ax.set_title("Sequence length distribution")
            ax.set_xlabel("Length (nt)"); ax.set_ylabel("Count")
            display(fig); plt.close(fig)

        except Exception as e:
            out_gen_txt.value = f"<pre style='color:red'>Error: {e}</pre>"

def on_selection(_):
    global ROUNDS
    with out_sel_fig:
        clear_output(wait=True)
        if not cb_enable_sel.value:
            out_sel_txt.value = "<pre>Selection disabled — skipping.</pre>"; ROUNDS=None; return
        if LIB_DF is None or LIB_DF.empty:
            out_sel_txt.value = "<pre style='color:red'>Generate the library first.</pre>"; return
        try:
            rounds       = int_rounds.value
            display_kind = dd_display.value
            S            = float_S.value
            C            = float_C.value
            cycles       = int_cycles.value
            bias_sig     = float_bias.value
            expr         = cb_expr.value
            expr_sig     = float_expr.value
            muKD, aKD, sKD = float_muKD.value, float_alphaKD.value, float_sigmaKD.value
            mols         = int_input.value
            seed         = int_seed_sel.value

            ROUNDS = selection_simulate_rounds(
                LIB_DF, rounds=rounds, display=display_kind, S=S, target_conc_nM=C,
                pcr_cycles=cycles, bias_sigma=bias_sig, expr_enabled=expr, expr_sigma=expr_sig,
                KD_mu_log10=muKD, KD_alpha=aKD, KD_sigma=sKD, input_molecules=mols, seed=seed
            )
            final = ROUNDS[-1]
            top5  = final.sort_values("frequency", ascending=False).head(5)
            out_sel_txt.value = f"<b>✅ Ran {rounds} round(s). Final pool size: {len(final)}.</b>" + render_df(
                top5[["seq_id","sequence","backbone_id","round","frequency","KD","capture_prob"]]
            )

            # Plots
            sh = []
            for rdf in ROUNDS:
                f = rdf["frequency"].to_numpy(); f = f / f.sum()
                sh.append(-(f * np.where(f>0, np.log(f), 0)).sum())
            figH, axH = plt.subplots(figsize=(6,3))
            axH.plot(range(1,len(ROUNDS)+1), sh, marker="o")
            axH.set_title("Diversity (Shannon) across rounds")
            axH.set_xlabel("Round"); axH.set_ylabel("Shannon H")
            display(figH); plt.close(figH)

            figT, axT = plt.subplots(figsize=(7,4))
            top_ids = final.sort_values("frequency", ascending=False).head(10)["seq_id"].tolist()
            for rid in top_ids:
                ys = []
                for rdf in ROUNDS:
                    row = rdf.loc[rdf["seq_id"] == rid]
                    ys.append(row["frequency"].values[0] if not row.empty else 0.0)
                axT.plot(range(1,len(ROUNDS)+1), ys, marker="o")
            axT.set_title("Top‑10 frequency trajectories")
            axT.set_xlabel("Round"); axT.set_ylabel("Frequency")
            display(figT); plt.close(figT)

            r1 = ROUNDS[0]
            figKD, axKD = plt.subplots(figsize=(6,3))
            axKD.hist(r1["KD"], bins=40); axKD.set_xscale("log")
            axKD.set_title("Round 1 affinity (KD)"); axKD.set_xlabel("KD"); axKD.set_ylabel("Count")
            display(figKD); plt.close(figKD)

            figCap, axCap = plt.subplots(figsize=(6,3))
            axCap.hist(r1["capture_prob"], bins=40)
            axCap.set_title("Round 1 capture probability"); axCap.set_xlabel("p_capture"); axCap.set_ylabel("Count")
            display(figCap); plt.close(figCap)

        except Exception as e:
            out_sel_txt.value = f"<pre style='color:red'>Error: {e}</pre>"

def on_ngs(_):
    global READS_DF
    with out_ngs_fig:
        clear_output(wait=True)
        if not cb_enable_ngs.value:
            out_ngs_txt.value = "<pre>NGS disabled — skipping.</pre>"; READS_DF=None; return
        if LIB_DF is None or LIB_DF.empty:
            out_ngs_txt.value = "<pre style='color:red'>Generate the library first.</pre>"; return
        try:
            src         = dd_ngs_source.value
            reads_total = int_reads.value
            p_err       = float_perr.value
            seed        = int_seed_ngs.value

            if src == "selection":
                if not ROUNDS:
                    out_ngs_txt.value = "<pre style='color:red'>Run selection first (or change source).</pre>"; return
                idx  = max(0, min(int_ngs_round.value-1, len(ROUNDS)-1))
                pool = ROUNDS[idx][["seq_id","sequence","frequency"]].copy()
                READS_DF = ngs_from_pool(pool, reads_total, p_err, seed=seed)
                src_label = f"selection round {idx+1}"
            else:
                pool = LIB_DF.copy(); pool["frequency"] = 1.0/len(pool)
                READS_DF = ngs_from_pool(pool[["sequence","frequency"]], reads_total, p_err, seed=seed)
                src_label = "true library (uniform)"

            out_ngs_txt.value = f"<b>✅ Simulated {len(READS_DF)} reads from {src_label}.</b>" + render_df(READS_DF)

            figE, axE = plt.subplots(figsize=(6,3))
            axE.hist(READS_DF["num_errors"], bins=20)
            axE.set_title("NGS per‑read error count")
            axE.set_xlabel("# errors per read"); axE.set_ylabel("Reads")
            display(figE); plt.close(figE)

        except Exception as e:
            out_ngs_txt.value = f"<pre style='color:red'>Error: {e}</pre>"

def on_abund(_):
    with out_ab_fig:
        clear_output(wait=True)
        try:
            model = dd_abund.value
            n = 5000
            rng = np.random.default_rng(123)
            if model == "uniform":
                weights = np.ones(n)
            elif model == "ln1":
                weights = rng.lognormal(mean=0.0, sigma=1.0, size=n)
            else:
                weights = rng.lognormal(mean=0.0, sigma=0.5, size=n)
            weights = weights / weights.sum()

            fig1, ax1 = plt.subplots(figsize=(6,3))
            ax1.hist(weights, bins=50)
            ax1.set_title(f"Abundance weights: {model}")
            ax1.set_xlabel("Weight"); ax1.set_ylabel("Count")
            display(fig1); plt.close(fig1)

            w_sorted = np.sort(weights)[::-1]
            topk = np.arange(1,51); cum = np.cumsum(w_sorted[:50])
            fig2, ax2 = plt.subplots(figsize=(6,3))
            ax2.plot(topk, cum, marker="o")
            ax2.set_title("Cumulative weight of top‑K sequences")
            ax2.set_xlabel("Top‑K"); ax2.set_ylabel("Cumulative fraction")
            display(fig2); plt.close(fig2)

            out_ab_txt.value = "Tip: log‑normal concentrates mass into few high‑abundance variants; uniform spreads it evenly."
        except Exception as e:
            out_ab_txt.value = f"<pre style='color:red'>Error: {e}</pre>"

# wire up
btn_gen.on_click(on_generate)
btn_sel.on_click(on_selection)
btn_ngs.on_click(on_ngs)
btn_abund.on_click(on_abund)

### 1) Library generation

### 2) Selection (display)

### 3) NGS simulation

### 4) Visualization: abundance skew (uniform vs. log‑normal)